# Backus Averaging Tutorial

This notebook demonstrates how to use **Backus averaging** to compute effective elastic properties of finely layered VTI (Vertically Transversely Isotropic) media.

## Theory

Backus (1962) averaging is used to calculate the long-wavelength effective elastic properties of a stack of thin layers. When the wavelength is much larger than the layer thickness, the layered medium behaves as a homogeneous VTI medium.

**Key Applications:**
- Shale-sand sequences
- Thinly bedded sediments
- Laminated rocks

**References:**
- Backus, G. E. (1962): Long-wave elastic anisotropy produced by horizontal layering
- Thomsen, L. (1986): Weak elastic anisotropy
- Mavko, G., Mukerji, T., & Dvorkin, J. (2009): The Rock Physics Handbook

In [ ]:
from drp_template.compute.rockphysics import backus_average, thomsen_params, vti_velocity_vs_angle
from drp_template.image import plot_velocity_vs_angle

## Import Required Functions

We'll use:
- `backus_average`: Calculate effective VTI elastic constants
- `thomsen_params`: Compute Thomsen anisotropy parameters (ε, γ, δ)
- `vti_velocity_vs_angle`: Calculate phase velocities as a function of propagation angle
- `plot_velocity_vs_angle`: Visualize velocity variations with angle

In [ ]:
# Define layer properties as arrays
# Layer 1: Calcite, Layer 2: Shale
Vp_layers = [5200, 2900]       # P-wave velocities (m/s)
Vs_layers = [2700, 1400]       # S-wave velocities (m/s)
rho_layers = [2450, 2340]      # Densities (kg/m³)
d_layers = [0.75, 0.5]         # Layer thicknesses (m)

# Set the angle (0 for vertical propagation, 90 for horizontal propagation)
angle = 90  # Change this to your desired angle

## Define Input Parameters

We'll model a two-layer system:
- **Layer 1:** Calcite (hard, high-velocity carbonate)
- **Layer 2:** Shale (soft, low-velocity clay-rich rock)

For each layer, we need:
- `Vp`: P-wave velocity (m/s)
- `Vshear`: S-wave velocity (m/s)
- `rho`: Density (kg/m³)
- `d`: Layer thickness (m)

In [ ]:
# Calculate Backus parameters using array-based API
results = backus_average(Vp_layers, Vs_layers, rho_layers, d_layers, angle_deg=angle, verbose=True)
M_ = results['M']
A_ = results['A']
D_ = results['D']
C_ = results['C']
B_ = results['B']
F_ = results['F']
Vp_ = results['Vp']
Vsv_ = results['Vsv']
Vsh_ = results['Vsh']
rho_eq = results['rho_eq']

# Calculate Thomsen parameters
thomsen = thomsen_params(A_, C_, F_, D_, M_)
print("\nThomsen Parameters:")
print(f"ε (epsilon) = {thomsen['epsilon']:.4f}")
print(f"γ (gamma)   = {thomsen['gamma']:.4f}")
print(f"δ (delta)   = {thomsen['delta']:.4f}")

## Calculate Backus Effective Properties

The `backus_average` function computes the five independent elastic constants for a VTI medium:

- **A (C₁₁)**: Horizontal P-wave modulus
- **C (C₃₃)**: Vertical P-wave modulus  
- **F (C₁₃)**: Anisotropic coupling parameter
- **D (C₄₄)**: Vertical shear modulus
- **M (C₆₆)**: Horizontal shear modulus

It also calculates:
- Effective density (ρ_eq)
- Phase velocities (Vp, Vsv, Vsh) at the specified angle
- Reference velocities at 0° and 90°

### Thomsen Parameters

The Thomsen parameters (ε, γ, δ) provide a convenient way to characterize weak anisotropy:
- **ε (epsilon)**: P-wave anisotropy
- **γ (gamma)**: S-wave anisotropy  
- **δ (delta)**: Controls near-vertical P-wave propagation

In [ ]:
# Calculate velocities vs angle
angles, Vp_array, Vsv_array, Vsh_array = vti_velocity_vs_angle(
    A=A_,
    C=C_,
    D=D_,
    F=F_,
    M=M_,
    rho_eq=rho_eq
)

## Calculate Phase Velocities vs Propagation Angle

Using the Christoffel equation, we compute how the three wave modes vary with propagation angle:

- **qP (Vp)**: Quasi-P-wave (predominantly compressional)
- **qSV (Vsv)**: Quasi-SV-wave (shear in the vertical plane)
- **SH (Vsh)**: Pure SH-wave (shear perpendicular to the vertical plane)

The angle ranges from:
- **0°**: Vertical propagation (along symmetry axis)
- **90°**: Horizontal propagation (perpendicular to symmetry axis)

In [ ]:
# Plot velocities vs angle
fig = plot_velocity_vs_angle(
    angles, Vp_array, Vsv_array, Vsh_array, 
    title='Wave Velocities for Fine Layered Medium',
    step_size=500
)

## Visualize Velocity Anisotropy

The plot shows how wave velocities change with propagation angle. Key observations:

- **P-wave anisotropy**: Difference between Vp(0°) and Vp(90°)
- **S-wave splitting**: Vsv ≠ Vsh in VTI media
- **Anisotropy strength**: Larger velocity variations indicate stronger anisotropy

The plot uses colorblind-friendly colors and adaptive y-axis scaling for clarity.

---

## Multi-Layer Example: Five Alternating Layers

The `backus_average_multilayer` function allows you to model sequences with **any number of layers**. This is useful for:

- Complex stratigraphic sequences
- Thinly interbedded formations
- Sensitivity studies with varying layer counts

Let's model a **5-layer alternating sequence** of sandstone and shale.

In [ ]:
# Define 5 alternating layers: Sandstone-Shale-Sandstone-Shale-Sandstone
n_layers_5 = 5

Vp_5layer = []
Vs_5layer = []
rho_5layer = []
d_5layer = []

for i in range(n_layers_5):
    if i % 2 == 0:  # Sandstone layers (0, 2, 4)
        Vp_5layer.append(4500)
        Vs_5layer.append(2800)
        rho_5layer.append(2500)
        d_5layer.append(0.8)
    else:  # Shale layers (1, 3)
        Vp_5layer.append(2900)
        Vs_5layer.append(1400)
        rho_5layer.append(2340)
        d_5layer.append(0.4)

print("5-Layer sequence:")
for i in range(n_layers_5):
    layer_type = "Sandstone" if i % 2 == 0 else "Shale"
    print(f"  Layer {i+1}: {layer_type:10s} - Vp={Vp_5layer[i]} m/s, Vs={Vs_5layer[i]} m/s, "
          f"ρ={rho_5layer[i]} kg/m³, d={d_5layer[i]} m")

### Now Let's Try a True Multi-Layer Sequence

Let's create a **5-layer alternating sequence** with different properties than the simple 2-layer case:

### Validation: 2-Layer Array Approach

The `backus_average` function now uses arrays for all layer properties, making it flexible for any number of layers. Let's verify it gives the same results for our 2-layer case:

In [ ]:
# Calculate effective properties for the 5-layer sequence
results_5layer = backus_average(
    Vp_5layer, Vs_5layer, rho_5layer, d_5layer,
    angle_deg=0,
    verbose=True
)

# Calculate Thomsen parameters
thomsen_5layer = thomsen_params(
    A=results_5layer['A'],
    C=results_5layer['C'],
    F=results_5layer['F'],
    D=results_5layer['D'],
    M=results_5layer['M']
)

print("\nThomsen Parameters (5-layer sequence):")
print(f"ε (epsilon) = {thomsen_5layer['epsilon']:.4f}")
print(f"γ (gamma)   = {thomsen_5layer['gamma']:.4f}")
print(f"δ (delta)   = {thomsen_5layer['delta']:.4f}")

# Calculate velocities vs angle for the 5-layer sequence
angles_5layer, Vp_5layer_angles, Vsv_5layer, Vsh_5layer = vti_velocity_vs_angle(
    A=results_5layer['A'],
    C=results_5layer['C'],
    D=results_5layer['D'],
    F=results_5layer['F'],
    M=results_5layer['M'],
    rho_eq=results_5layer['rho_eq']
)

# Plot
fig = plot_velocity_vs_angle(
    angles_5layer, Vp_5layer_angles, Vsv_5layer, Vsh_5layer,
    title='Wave Velocities: 5-Layer Sandstone-Shale Sequence',
    step_size=200
)